# GROUP ASSIGNMENT [COSC2753]

**Members**:
- Vo Thanh Luan – s3822042

# I. Problem statement

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2

In [26]:
import os
for dirname, _, filenames in os.walk('./data/Flowers'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/Flowers/.DS_Store
./data/Flowers/Babi/babi_926.jpg
./data/Flowers/Babi/babi_729.jpg
./data/Flowers/Babi/babi_701.jpg
./data/Flowers/Babi/babi_715.jpg
./data/Flowers/Babi/babi_259.jpg
./data/Flowers/Babi/babi_265.jpg
./data/Flowers/Babi/babi_503.jpg
./data/Flowers/Babi/babi_517.jpg
./data/Flowers/Babi/babi_271.jpg
./data/Flowers/Babi/babi_488.jpg
./data/Flowers/Babi/babi_12.jpg
./data/Flowers/Babi/babi_339.jpg
./data/Flowers/Babi/babi_311.jpg
./data/Flowers/Babi/babi_477.jpg
./data/Flowers/Babi/babi_463.jpg
./data/Flowers/Babi/babi_305.jpg
./data/Flowers/Babi/babi_852.jpg
./data/Flowers/Babi/babi_846.jpg
./data/Flowers/Babi/babi_649.jpg
./data/Flowers/Babi/babi_891.jpg
./data/Flowers/Babi/babi_885.jpg
./data/Flowers/Babi/babi_675.jpg
./data/Flowers/Babi/babi_113.jpg
./data/Flowers/Babi/babi_107.jpg
./data/Flowers/Babi/babi_661.jpg
./data/Flowers/Babi/babi_106.jpg
./data/Flowers/Babi/babi_660.jpg
./data/Flowers/Babi/babi_674.jpg
./data/Flowers/Babi/babi_112.jpg
./data/Flowers/Babi

# II. Data retrieving

In [34]:
data_dir = "./data/Flowers"
flower_folders = os.listdir(data_dir)
print(flower_folders)

['Babi', 'Pingpong', 'Chrysanthemum', 'Rosy', 'Tana', 'Hydrangeas', 'Lisianthus', 'Calimerio']


In [35]:
link = []
for label in flower_folders:
    path = os.path.join(data_dir, label) # combine path and labels
    link.append(path) # append in link
print(link)


for i in range(len(link)):
    new = os.listdir(link[i])
    i+=1
    print(f"length : {len(new)}") # each folder total image count

['./data/Flowers/Babi', './data/Flowers/Pingpong', './data/Flowers/Chrysanthemum', './data/Flowers/Rosy', './data/Flowers/Tana', './data/Flowers/Hydrangeas', './data/Flowers/Lisianthus', './data/Flowers/Calimerio']
length : 931
length : 360
length : 696
length : 171
length : 623
length : 518
length : 969
length : 353


In [ ]:
IMG_SIZE = 224 # image size

data = []

def get_data(data_dir):
    for category in labels:
        path = os.path.join(data_dir, category) #combine path
        class_num = labels.index(category) # index no of labels
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR) # color image array
                resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) # resize
                data.append([resized_array, class_num])
            except Exception as e: # exception
                print(e)
    return np.array(data)#%% md
# GROUP ASSIGNMENT [COSC2753]

**Members**:
- Vo Thanh Luan – s3822042

# I. Problem statement

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2

In [ ]:
import os
for dirname, _, filenames in os.walk('./data/Flowers'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# II. Data retrieving

In [ ]:
data_dir = "./data/Flowers"
flower_folders = os.listdir(data_dir)
print(flower_folders)

In [ ]:
link = []
for label in flower_folders:
    path = os.path.join(data_dir, label) # combine path and labels
    link.append(path) # append in link
print(link)

for i in range(len(link)):
    new = os.listdir(link[i])
    i+=1
    print(f"length : {len(new)}") # each folder total image count

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (4621, 2) + inhomogeneous part.

array(['Babi', 'Pingpong', 'Chrysanthemum', 'Rosy', 'Tana', 'Hydrangeas',
       'Lisianthus', 'Calimerio'], dtype=object)